In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#install packages
!pip install -r https://github.com/casperbh96/Web-Scraping-Reddit/raw/master/requirements.txt

     |████████████████████████████████| 99 kB 2.9 MB/s eta 0:00:011
     |████████████████████████████████| 101 kB 6.1 MB/s eta 0:00:01
     |████████████████████████████████| 158 kB 13.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 16.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 47.8 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 4.4 MB/s  eta 0:00:01
     |████████████████████████████████| 294 kB 36.2 MB/s eta 0:00:01
     |████████████████████████████████| 89 kB 7.3 MB/s  eta 0:00:01
     |████████████████████████████████| 5.7 MB 24.1 MB/s eta 0:00:01
     |████████████████████████████████| 87 kB 5.6 MB/s  eta 0:00:01
     |████████████████████████████████| 12.9 MB 31.0 MB/s eta 0:00:01
     |████████████████████████████████| 13.8 MB 52.1 MB/s eta 0:00:01
     |████████████████████████████████| 10.4 MB 38.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 37.7 MB/s eta 0:00:01
     |██████████████████████████████

      Successfully uninstalled six-1.14.0
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.9.0
    Uninstalling absl-py-0.9.0:
      Successfully uninstalled absl-py-0.9.0
  Attempting uninstall: soupsieve
    Found existing installation: soupsieve 1.9.4
    Uninstalling soupsieve-1.9.4:
      Successfully uninstalled soupsieve-1.9.4
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.9.0
    Uninstalling beautifulsoup4-4.9.0:
      Successfully uninstalled beautifulsoup4-4.9.0
  Attempting uninstall: certifi
    Found existing installation: certifi 2020.6.20
    Uninstalling certifi-2020.6.20:
      Successfully uninstalled certifi-2020.6.20
  Attempting uninstall: gast
    Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.30.0
    Uninstalling grpcio-1.30.0:
      Successfully uninstalled g

Generally, web scraping involves accessing numerous websites and collecting data from them. However, we can limit ourselves to collect large amount of information from a single source and use it as a dataset.

We’ll access the List of countries and dependencies by population Wikipedia webpage. The webpage includes a table with the names of countries, their population, date of data collection, percentage of world population and source. And if we go to any country’s page, all information about it is written on the page with a standard box on the right. This box includes a lot of information such as total area, water percentage, GDP etc.
Here, we will combine the data from these two webpages into one dataset.
List of Countries: On accessing the first page, we’ll extract the list of countries, their population and percentage of world population.
Country: We’ll then access each country’s page, and get information including total area, percentage water, and GDP (nominal).
Thus, our final dataset will include information about each country.

In [1]:
#import libs
import numpy as np
import pandas as pd

from urllib.request import urlopen
from bs4 import BeautifulSoup

Initially, we define just the basic function of reading the url and then extracting the HTML from the same.

In [2]:
def getHTMLContent(link):
    html = urlopen(link)
    soup = BeautifulSoup(html, 'html.parser')
    return soup

we use the find_all() method to find all tables on the page. The parameter that we supply inside this function determines the element that it returns. As we require tables, we pass the argument as table and then iterate over all tables to identify the one we need.

In [3]:
content = getHTMLContent('https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population')
tables = content.find_all('table')
for table in tables:
    print(table.prettify())

<table class="wikitable sortable" style="text-align:right">
 <tbody>
  <tr>
   <th data-sort-type="number">
    Rank
   </th>
   <th>
    Country (or dependent territory)
   </th>
   <th>
    Population
   </th>
   <th>
    % of world
    <br/>
    population
   </th>
   <th>
    Date
   </th>
   <th class="unsortable">
    Source
   </th>
  </tr>
  <tr>
   <th>
    1
   </th>
   <td align="left">
    <span class="flagicon">
     <img alt="" class="thumbborder" data-file-height="600" data-file-width="900" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/23px-Flag_of_the_People%27s_Republic_of_China.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/35px-Flag_of_the_People%27s_Republic_of_China.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/45px-Flag_of_the_People%27s_Republic_of_C

Now, we need to analyse the output and see which table has the data we are searching for. After much inspection, we can see that the table with the class, wikitable sortable, has the data we need. 

In [7]:
table = content.find('table', {'class': 'wikitable sortable'})
rows = table.find_all('tr')

# List of all links
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 1:
        country_link = cells[1].find('a')
       # print(country_link.get('href'))

In [8]:
def getAdditionalDetails(url):
    try:
        country_page = getHTMLContent('https://en.wikipedia.org' + url)
        table = country_page.find('table', {'class': 'infobox geography vcard'})
        additional_details = []
        read_content = False
        for tr in table.find_all('tr'):
            if (tr.get('class') == ['mergedtoprow'] and not read_content):
                link = tr.find('a')
                if (link and (link.get_text().strip() == 'Area' or
                   (link.get_text().strip() == 'GDP' and tr.find('span').get_text().strip() == '(nominal)'))):
                    read_content = True
                if (link and (link.get_text().strip() == 'Population')):
                    read_content = False
            elif ((tr.get('class') == ['mergedrow'] or tr.get('class') == ['mergedbottomrow']) and read_content):
                additional_details.append(tr.find('td').get_text().strip('\n')) 
                if (tr.find('div').get_text().strip() != '•\xa0Total area' and
                   tr.find('div').get_text().strip() != '•\xa0Total'):
                    read_content = False
        return additional_details
    except Exception as error:
        print('Error occured: {}'.format(error))
        return []

Iterating over all rows in the table, the iterator checks if the present row is a heading matching Area or GDP (nominal) and starts reading. In reading mode, it checks if the new element is Total area or Total and if yes, it reads it and keeps reading to read the Water (%) or per capita GDP respectively in the next run.

We use try and except to ensure that even if we miss certain values, our whole process does not end. 

Create the dataset

In [ ]:
data_content = []
for row in rows:
    cells = row.find_all('td')
    if len(cells) > 1:
        print(cells[1].get_text())
        country_link = cells[1].find('a')
        country_info = [cell.text.strip('\n') for cell in cells]
        additional_details = getAdditionalDetails(country_link.get('href'))
        if (len(additional_details) == 4):
            country_info += additional_details
            data_content.append(country_info)

dataset = pd.DataFrame(data_content)

# Define column headings
headers = rows[0].find_all('th')
headers = [header.get_text().strip('\n') for header in headers]
headers += ['Total Area', 'Percentage Water', 'Total Nominal GDP', 'Per Capita GDP']
dataset.columns = headers

drop_columns = ['Rank', 'Date', 'Source']
dataset.drop(drop_columns, axis = 1, inplace = True)
dataset.sample(3)

dataset.to_csv("Dataset.csv", index = False)